# Remote-Sensing Project

## Import Libraries

aaaaaaaaaaaaaaa<cite data-cite="perez2011python">(Granger, 2013)</cite>
    


In [1]:
# Project RemoteSensing
# first lets insert the libraries
# We get the data through the google earth engine
# Which is a cloud that has all available images Landsat
# Basic libaries
import pandas as pd
import geopandas as gpd
# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

# next we need to authenticate Earth Engine
ee.Authenticate()

# Initialize the library
ee.Initialize()



ModuleNotFoundError: No module named 'fcntl'

## Define filters

In [ ]:
# coords of Chalkidiki fire
lat = 40.116642
lon = 23.949377

# point of interest(poi) as an ee.Geometry (with lat and lot)
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter at
# (landsat is available every 8 cal dates)
# not a good image though every 8 days, means
# we need more data=bigger date span
start_date = '2018-06-01'

# end date to filter at
end_date = '2018-12-01'

## Get Landsat 8 Data

In [ ]:
# get the landsat8-satellite Data
# Landsat has a lot of info
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
    `        .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [ ]:
# how many images did we actually get?
# adgust the time span
print('Total number:', landsat.size().getInfo())

In [ ]:
# information about the first image in our collection
# when was it taken 
landsat.first().getInfo()

In [ ]:
# lets get the cloud cover first
landsat.first().get('CLOUD_COVER').getInfo()

In [ ]:
# lets see when it was acquired
landsat.first().get('DATE_ACQUIRED').getInfo()

In [ ]:
# different wavelengths are different data
# lets see the bands we got
landsat.first().bandNames().getInfo()

In [ ]:
# bring the images and list them so we can loop and output them
landsat_list = landsat.toList(landsat.size())

## Display satellite image

In [ ]:
# set parameters for images(RGB)
parameters = { 'min':7000, 'max':18000, 'dimensions':800,
                 'bands':['SR_B2','SR_B3','SR_B4'] # RGB colours
             }

In [ ]:
# create data
data = []

# loop and display images
for i in range(landsat.size().getInfo()):
    
    # 1. When was it taken
    date =  ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # 2. What the cloud cover is
    cloud_cover = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # 3. Print the image label
    print('Image #',i,date,'Cloud cover:',cloud_cover)
    
    # 4. Display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))
    
    # 5. List data
    data_ = [i, date, cloud_cover]
    
    # 6. Append
    data.append(data_)
    
    
# create pandas dataframe(like excel)
p_df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

In [ ]:
# our data in table
p_df

##  Choose Images

In [ ]:
# lets pick a list of images
# that we want to search further
landsat_sequence = [5,7,9]

In [ ]:
# Define the region by the poi
# radius is chosen
roi = poi.buffer(10000)

In [ ]:
# set parameters for images(RGB)
parameters = { 'min':7000, 'max':18000, 'dimensions':800,
                 'bands':['SR_B2','SR_B3','SR_B4'], # RGB colours
                 'region':roi
             }

In [ ]:
for i in landsat_sequence:
    
    # 1. Time
    date =  ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # 2. Cloud cover 
    cloud_cover = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # 3. Print the image label
    print('Image #', i, date,'Cloud cover', cloud_cover)
    
    # 4. Display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))


## Normalized Difference Vegetation Index
cite this <cite data-cite="Kaimaris2014RS"></cite>

In [ ]:
# ndvi parameters: red is low, green is high vegetation values
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0, # red = 0
                   'max': 0.4, # green >=0.4
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [ ]:
for i in landsat_sequence:
    
    # 1. Time
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # 2. Cloud cover 
    cloud_cover = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # 3. Print the image label
    print('Image #', i, date,'Cloud cover', cloud_cover)
    
    # 4. Display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

## Folium
Visualising NDVI version of image in interactive mapping environment

In [ ]:
# folium map
import folium
# Create a javascript leaflet map
m = folium.Map(location=[lat,lon],zoom_start=13)
m

In [ ]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Create a folium map
int_map = folium.Map(location=[lat, lon], zoom_start=13)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    int_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(int_map)

# Add lon lat coords user defined
int_map.add_child(folium.LatLngPopup())

folium.Circle(
    radius=250,
    location=[40.1244, 23.9380],
    color="purple",
    fill=False,
).add_to(int_map)

folium.Marker(
    location=[40.1244, 23.9380],
    popup="Possible start of fire due to ΔΕΗ column falling",
    icon=folium.Icon(color="purple", icon="info-sign"),
).add_to(int_map)

folium.Circle(
    radius=500,
    location=[40.097474, 23.977585],
    popup="Sarti",
    color="blue",
    fill=False,
).add_to(int_map)

folium.Marker(
    location=[40.097474, 23.977585],
    popup="Sarti",
    icon=folium.Icon(color="blue", icon="info-sign"),
).add_to(int_map)

folium.Marker(
    location=[40.0984, 23.9428],
    popup="fiery front",
    icon=folium.Icon(color="green", icon="stop"),
).add_to(int_map)

folium.Marker(
    location=[40.1030, 23.9572],
    popup="fiery front",
    icon=folium.Icon(color="green", icon="stop"),
).add_to(int_map)

folium.Marker(
    location=[40.1041, 23.9702],
    popup="fiery front",
    icon=folium.Icon(color="green", icon="stop"),
).add_to(int_map)

folium.Marker(
    location=[40.1054, 23.9804],
    popup="fiery front",
    icon=folium.Icon(color="green", icon="stop"),
).add_to(int_map)

# Display the map.
display(int_map)

In [ ]:
int_map.save('Chalkidiki_fire.html')

## Bibliography
[1] geagerageargearg \
[2] aegaergerg \
[3] aegaergaergaergae \
[4] eargaergerg \



---
title: "Sample Document"
output: pdf_document
bibliography: bibliography.bib
csl: biomed-central.csl
---
title: "Sample Document"
output: pdf_document
bibliography: bibliography.bib
csl: biomed-central.csl
---


In [ ]:
 #automatic document conversion to markdown and then to word
 #first convert the ipython notebook paper.ipynb to markdown
 os.system("jupyter nbconvert --to markdown RSmain.ipynb")
 #next convert markdown to ms word
 conversion <- paste0("pandoc -s RSmain.md -t docx -o RSmain.docx",
                " --citeproc",
                " --bibliography=Ref.bib",
                " --csl=ieee.csl")
 os.system(conversion)